In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import imageio.v3 as iio

In [2]:
data = [[0,0,0,0,0,0,0,0]]
data = pd.DataFrame(data, columns =["UID","Specie","SubSpecie","Distribution","href","Genus","Genus_Info","Value"])

In [3]:
data.to_csv("data/orchids.csv",index=False, header=True)

In [4]:
base_url = "https://www.worldplants.de/world-orchids/orchid-list"
r = requests.get(base_url)
# print request object
print(r.url)

# print status code
print(r.status_code)

https://www.worldplants.de/world-orchids/orchid-list
200


In [5]:
# Parsing the HTML
soup = BeautifulSoup(r.content, 'html.parser')
 
s = soup.find('select', id='genus')
results = s.find_all('option')
genus = []
values = []
for result in results:
    genus.append(result.text)
    values.append(result.get('value'))

genus.remove("...")
values.remove("0")

In [6]:
# we generate the url
url = base_url + "#g-"+values[300]
browser = webdriver.Chrome()

browser.get(url)
wait = WebDriverWait(browser,20)
html = browser.page_source


In [21]:
# We get distribution information on the Genus
genus_info = browser.find_elements(By.CSS_SELECTOR, '.genusInfo')
dis = []
genus_distribution = ""
for e in genus_info:
    dis = e.text.split("\n")
for i in dis:
    if "Distribution:" in i:
        genus_distribution = i

In [22]:
genus_distribution

'Distribution: S-China, tropical Asia to Pacific. This is a major spinoff genus from Eria s. lat. The separation is supported by molecular results.'

In [23]:
#Let's try an alternative method to retrieve the sub-species
orquid_selection = browser.find_elements(By.CSS_SELECTOR, ".type-species")
name = []
dist = []
href = []
h = []
data = []
distribution =[]
for i in orquid_selection:
    u_id =int(i.get_attribute("id").split("-")[1])
    n = i.find_element(By.CSS_SELECTOR, ".latName").text
    typ = "specie"
    name.append([u_id,n,typ])
    tmp = i.find_elements(By.CSS_SELECTOR, ".distCountry")
    for j in tmp:
        dist.append(j.text)
    distribution.append(dist)
    dist = []
    image = i.find_elements(By.CSS_SELECTOR, "div.imageBox a")
    #print(image)
    for j in image:
        tmp =j.get_attribute("href")
        if tmp not in h:
            h.append(tmp)    
    href.append([u_id,h])
    h = []
    data.append([u_id,n,distribution,h])

In [24]:
orchid_sub = browser.find_elements(By.CSS_SELECTOR, ".type-subspecies")
name_sub = []
dist_sub = []
href_sub = []
h = []
data_sub = []
distribution = []
for i in orchid_sub:
    u_id =int(i.get_attribute("id").split("-")[1])
    n = i.find_element(By.CSS_SELECTOR, ".latName").text
    typ = "sub-specie"
    name_sub.append([u_id,n,typ])
    tmp = i.find_elements(By.CSS_SELECTOR, ".distCountry")
    for j in tmp:
        dist_sub.append([j.text])
    distribution.append(dist_sub)
    dist_sub = []
    image = i.find_elements(By.CSS_SELECTOR, "div.imageBox a")
    #print(image)
    for j in image:
        tmp =j.get_attribute("href")
        if tmp not in h:
            h.append(tmp)
    href_sub.append(h)
    data_sub.append([u_id,n,distribution,h])
    h = []

In [25]:
data = pd.DataFrame(data, columns =["UID","Specie","Distribution","href"])
data["SubSpecie"] = ""
data.head()

,UID,Specie,Distribution,href,SubSpecie
0,55251,Pinalia acervata,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],
1,55252,Pinalia acutissima,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[https://www.worldplants.de/fileadmin/Modules/...,
2,55253,Pinalia affinis,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[https://www.worldplants.de/fileadmin/Modules/...,
3,55254,Pinalia amica,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],
4,55255,Pinalia amplectens,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],


In [26]:
data_sub = pd.DataFrame(data_sub, columns =["UID","SubSpecie","Distribution","href"])
data_sub["Specie"] = ""
data_sub.head()

,UID,SubSpecie,Distribution,href,Specie


In [27]:
j = 0
k = 0
for i in name_sub:
    while k == 0:
        if (i[0]-j) in data["U-ID"].values:
            spe = data[data["U-ID"] == (i[0]-j)]["Specie"].unique()
            data_sub.loc[data_sub["U-ID"]==i[0],"Specie"] = spe
            k = 1
        j += 1
    j = 0
    k = 0    


In [28]:
data = pd.concat([data,data_sub],ignore_index = True)
data.head()

,UID,Specie,Distribution,href,SubSpecie
0,55251,Pinalia acervata,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],
1,55252,Pinalia acutissima,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[https://www.worldplants.de/fileadmin/Modules/...,
2,55253,Pinalia affinis,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[https://www.worldplants.de/fileadmin/Modules/...,
3,55254,Pinalia amica,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],
4,55255,Pinalia amplectens,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],


In [29]:
data["Genus"] = genus[300]
data["Value"] = values[300]
data["Genus_Info"] = genus_distribution

In [30]:
data.head()

,UID,Specie,Distribution,href,SubSpecie,Genus,Value,Genus_Info
0,55251,Pinalia acervata,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],,Pinalia,1366,"Distribution: S-China, tropical Asia to Pacifi..."
1,55252,Pinalia acutissima,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[https://www.worldplants.de/fileadmin/Modules/...,,Pinalia,1366,"Distribution: S-China, tropical Asia to Pacifi..."
2,55253,Pinalia affinis,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[https://www.worldplants.de/fileadmin/Modules/...,,Pinalia,1366,"Distribution: S-China, tropical Asia to Pacifi..."
3,55254,Pinalia amica,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],,Pinalia,1366,"Distribution: S-China, tropical Asia to Pacifi..."
4,55255,Pinalia amplectens,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],,Pinalia,1366,"Distribution: S-China, tropical Asia to Pacifi..."


In [31]:
data = data[["UID","Specie","SubSpecie","Distribution","href","Genus","Genus_Info","Value"]]

In [32]:
data.head()

,UID,Specie,SubSpecie,Distribution,href,Genus,Genus_Info,Value
0,55251,Pinalia acervata,,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],Pinalia,"Distribution: S-China, tropical Asia to Pacifi...",1366
1,55252,Pinalia acutissima,,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[https://www.worldplants.de/fileadmin/Modules/...,Pinalia,"Distribution: S-China, tropical Asia to Pacifi...",1366
2,55253,Pinalia affinis,,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[https://www.worldplants.de/fileadmin/Modules/...,Pinalia,"Distribution: S-China, tropical Asia to Pacifi...",1366
3,55254,Pinalia amica,,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],Pinalia,"Distribution: S-China, tropical Asia to Pacifi...",1366
4,55255,Pinalia amplectens,,"[[China, SE-Tibet, Nepal, India, Bhutan, Laos,...",[],Pinalia,"Distribution: S-China, tropical Asia to Pacifi...",1366


<h3>Code To Generate the Images

In [33]:
for i in data.index:
    m = data["href"][i]
    name = data["Specie"][i]
    if len(m) != 0:
        for j,k in enumerate(m):             
            tmp = iio.imread(k)
            iio.imwrite(f"images/{name}{j}.jpg",tmp);

<h3>Save the data to a csv file

In [35]:
data.to_csv('data/orchids.csv',mode='a', index=False, header=False)